# ARTG Scraper (Rewrite)

## Introduction

Recently, I wrote a scraper for the "Breaking Good" project (hereafter BG), a project which aims to collect information about the accessibility and availability of medicines classified as "essential" by the W.H.O.

At the time, I had written the tool using Selenium - largely because I had little experience with BeautifulSoup, and I found it relatively-easy to implement automated file downloads using the former. Having now spent a bit of time getting aquainted with the latter, and because I am right now on holiday and the scope of the scraper requirements is clearer, I figured it might be time to re-visit the project, for fun, and to see if I can glean any new insights from the data.

#### Note for future self: 

<b>Every ARTG entry should have a "Public ARTG summary" PDF - pdfplumber looks like it does a very good job of parsing these documents, and the ones I've tested have all had pretty consistent formats, from 2001 to 2021. These documents contain lots of pertinent data!</b>

## Datasets

A list of essential medicines of particular interest to the BG team was provided as a single-column CSV. We can plug the medicines on that list into the Aus. Govt. Australian Register of Therapuetic Goods (ARTG), hosted by the EBS. From that, we should be able to get a list of relevant ARTG IDs, which are unique identifiers for each medicine and its formulations.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import pdfplumber
import re
import time
import urllib3
import io

In [60]:
## Load the original CSV of essential medicine names
essential_medicine_names = pd.read_csv(f"WHO_essential_anti_infective_medicines.csv", header=None)

## what is the dimensionality of the longest search string? (We will need it later)
essential_medicine_names_split = pd.concat([essential_medicine_names[0].str.split(', ', expand=True)], axis=1)
essential_medicine_names_split.head()
num_cols = essential_medicine_names_split.shape[1] ## in this case, 4

## get rows where column 3 is not None (just to see)
non_none = essential_medicine_names_split.loc[essential_medicine_names_split[num_cols-1].notnull(), [i for i in range(0,num_cols)]]
non_none.head()

,0,1,2,3
62,ethambutol,isoniazid,pyrazinamide,rifampicin
74,isoniazid,pyridoxine,sulfamethoxazole,trimethoprim


In [410]:
def formatted_active_ingredient_string(input_string):
    '''
    Receives a comma-separated string of non-zero length containing medicine names:
        "abacavir" OR "abacavir, lamivudine" etc.
    
    Returns a formatted string for the meta_A tag on the TGA search client url:
        "abacavir" OR "abacavir%2C+lamivudine"
        
        If an empty string is given as input, function returns 0.
    '''
    
    if input_string == "" or input_string == " ":
        return 0
    
    ## convert string to list
    input_string = input_string.split(',')
    
    ## first active ingredient is first element
    actives = str(input_string[0])
    
    ## if there is more than one active ingredient, 
    ## append it to formatted string
    if len(input_string) > 1:
        for ing in [str(ing.lstrip()) for ing in input_string[1:]]:
            actives+="%2C+"+str(ing)
    
    return actives


def artg_search(*args, **kwargs):
    '''
    Performs a search of the ARTG, in two modes. First, if a formatted ingredient string is passed
    from formatted_active_ingredient_string and a start_rank integer is also passed, we are 
    running a search of the ARTG on the main database page. Otherwise, if we receive only
    a valid ARTG identifier, we are running a sub-search of the database for a particular medicine.
    These two modes have different URLs, and will return Soup objects that need to be parsed differently.
    
    Returns a BeautifulSoup object.
    '''
    
    if "formatted_string" in kwargs and "start_rank" in kwargs:
        url = f"https://tga-search.clients.funnelback.com/s/search.html?from-advanced=true&collection=tga-artg&meta_A={kwargs['formatted_string']}&fmo=on&start_rank={kwargs['start_rank']}"
    elif "artg_id" in kwargs:
        url = f"https://tga-search.clients.funnelback.com/s/search.html?collection=tga-artg&profile=record&meta_i={kwargs['artg_id']}"
    else: return 0
    
    ## check status code is 200
    req = requests.get(url)
    if req.status_code != 200:
        return 0
    
    return BeautifulSoup(req.text, 'lxml')

def read_pdf_from_url(url):
    '''
    Code taken from Demian Wolf @ 
    https://stackoverflow.com/questions/62075033/read-pdf-from-url-to-memory-omitting-saving-file-to-local-file &&
    '''
    http = urllib3.PoolManager()
    temp = io.BytesIO()
    temp.write(http.request("GET", url).data)

    all_text = '' # new line
    with pdfplumber.open(temp) as pdf:
        
        for pdf_page in pdf.pages:
            
            # separate each page's text with newline
            single_page_text = pdf_page.extract_text()
            all_text = all_text + '\n' + single_page_text
            
    return all_text


def get_ARTG_data_from_string(query_string, category):
    '''
    Does what it says on the box. Receives a query_string (ideally, the PDF contents from read_pdf_from_url)
    and returns a match for "category XXX", where XXX is the value of category, if it exists.
    
    Valid categories include:
    * Start Date
    * Dosage Form
    * Route of Administration
    * Effective Date
    * Status (we might not keep this one, if it's not interesting)
    '''
    
    date_matches = ["Start Date", "Effective Date"]
    pair_matches = ["Dosage Form", "Route of Administration", "Status"]
    
    match_string = ""
    if category in date_matches:
        match_string = f"({category}) (\d+/\d+/\d+)"
    elif category in pair_matches:
        match_string = f"({category}) (.+)"
    else:
        return np.nan
    
    match = re.search(match_string, query_string)
    try:
        return [match.group(2).lstrip().rstrip()]
    ## If there is no match in the string, group will fail on None type
    except AttributeError:
        return np.nan
    
def get_number_of_subsearches(soup):
    '''
    Receives a beautifulsoup object. Finds the "description" class, and returns substrings that
    match the regex string.
    
    Check that search returned zero partial matches, and use the first group to define 
    the number of sub-searches we need to make (due to pagination).
    '''
    
    description_string = soup.find("p", {"class": "description"}).text
    match = re.search(r'Documents: (\d+) fully matching plus (\d+) partially matching', description_string)
    
    return int(match.group(1)), int(match.group(2))


def convert_profile_table(ID_string, profile_soup):

    ## get the profile table into a dict; initialise the table with known ID; init variables
    public_summary_data_OI = ["Start Date", "Dosage Form", "Route of Administration", "Effective Date", "Status"]
    keys_not_of_interest = ["Product Information", "Public ARTG summary", "Consumer Medicines Information", "ARTG entry for"]
    table_dict = {"ARTG ID": int(ID_string)}
    table = profile_soup.find('table')

    for row in table.findAll('tr'):
        if row.find('th').text == "Public ARTG summary":
            table_dict[row.find('th').text] = row.find('a')['href']
        else:
            table_dict[row.find('th').text] = [row.find('td').text]
    
    if table_dict["Public ARTG summary"]:
        public_summary_text = read_pdf_from_url(table_dict["Public ARTG summary"])
        
        for datum in public_summary_data_OI:
            table_dict[datum] = get_ARTG_data_from_string(public_summary_text, datum)
            
        ## and get strengths (using a different function) ## this will now be a dict
        strength_mask = get_ingredients_and_strengths_from_PDF(public_summary_text)
        
        #print(f'hello: {strength_mask}')

    
    ## we also want to check that, if we've done a search for M ingredients, that the dimensions of the
    ## active ingredient value is equal to M, and that each of the M ingredients appears in the value
    ## i.e. if we search for "lamivudine, amoxicillin", we want to ensure we only get that combination,
    ## not one in which there are those two PLUS an additional drug.
    
    
    ## we should also add N columns to the dataframe, where N is the largest size of all the active ingredients
    ## we want this because then each ingredient is given it's own column, and each ingredient strength has its own
    ## column -- is this a good idea? i.e.
    ## Ingredient 1: lamivudine; Ingredient 1 Strength: XXX (units); Ingredient 2: amoxicillin; Ingredient 2 Strength: XXX (units)

    for key in keys_not_of_interest:
        table_dict.pop(key, None)

    return pd.DataFrame.from_dict(table_dict)

def get_ingredients_and_strengths_from_PDF(soup):
    '''
    This will regex all lines between (and including) "Active Ingredients" and "Other Ingredients" in the
    PDF soup returned from read_pdf_from_url. It is likely that this will occasionally span across pages,
    so we will need to implement cleaning!
    '''

    out_dict = {}
    
    words_to_remove = ['Equivalent', 'AEST', 'ARTG Certificate', 'onus', 'www.tga.gov.au', 'Commonwealth'] ## empirically-derived
    ## returns lines as per above. EOF version used when no "Other Ingredients" section
    active_ingredients_lines = re.findall("[^\n]*Active\ Ingredients.*?Other\ Ingredients[^\n]*", soup, re.DOTALL)

    
    ### This can be complicated - what if there is more than one match for active_ingredients_lines?
    ### furthermore, what if we match N times, but the last one doesn't have "Other Ingredients" at the end?
    ### We need to treat the matches as a list, but then also consider that there might be more than one way
    ### of getting a match...
    
    ### A problem for another day, I think.
    
    if active_ingredients_lines != []:
        #for i in range(len(active_ingredients_lines)):
        #       print(test[i].split("\n")[1:-1])
        #### NOTE: we need to treat this like a list, !!!!
        active_ingredients_lines = active_ingredients_lines[0].split("\n")[1:-1]
    else:
        active_ingredients_lines = re.findall("[^\n]*Active\ Ingredients.*? (?:).*", soup, re.DOTALL)
        active_ingredients_lines = active_ingredients_lines[0].split("\n")[1:]
        
    active_ingredients_lines = [a for a in active_ingredients_lines if not any(b in a for b in words_to_remove)]
    active_ingredients_lines = [a.lstrip().rstrip() for a in active_ingredients_lines if not len(a.lstrip().rstrip()) == 1 and not len(a.lstrip().rstrip()) == 0]
    
    
    pattern = '(^\D*)([0-9]*\.?[0-9]+.*[a-zA-Z]?.*[a-zA-Z]+$)'
    for ingredient_line in active_ingredients_lines:
        match = re.search(pattern, ingredient_line, flags=re.I)
        try:
            ## ingredient, strength
            out_dict[match.group(1).rstrip()] = match.group(2)
        except AttributeError:
            pass
    
    return out_dict




In [396]:
out_df = pd.DataFrame()

#for i in range(len(essential_medicine_names)):   
for i in range(8,9):
    rank = 1
    medicine = essential_medicine_names.iloc[i, 0]
    f_string = formatted_active_ingredient_string(medicine)
    ini_soup = artg_search(formatted_string=f_string, start_rank=rank)
    
    full_matches, partial_matches = get_number_of_subsearches(ini_soup)
    if partial_matches != 0 or full_matches == 0:
        continue
    
    while rank < full_matches:

        ## get the results page
        soup = artg_search(formatted_string=f_string, start_rank=rank)
        
        ## find all identifiers on the page
        ARTD_ID_list = [x.text for x in soup.findAll('li') if 'ARTG ID:' in str(x.text)]
        
        ## for each ID, run a profile search
        for ID_fullstring in ARTD_ID_list:
            
            ID_string = re.search(r'\d+', ID_fullstring).group(0)
            profile_soup = artg_search(artg_id=ID_string)
            current_df = convert_profile_table(ID_string, profile_soup)
            out_df = out_df.append(current_df, ignore_index=True)
        
        rank += 10 ## pagination increment by 10
#        print(f"Getting soup for medicine: {medicine}; nmeds: {full_matches}, rank: {rank}")


['Active Ingredients\namoxicillin trihydrate 1148 mg\nEquivalent: amoxicillin 1000 mg\nOther Ingredients (Excipients)']
['Active Ingredients\namoxicillin trihydrate 137.71 mg/mL\nEquivalent: amoxicillin 120 mg/mL\npotassium clavulanate 10.28 mg/mL\nEquivalent: clavulanic acid 8.58 mg/mL\nOther Ingredients (Excipients)']
['Active Ingredients\nesomeprazole magnesium trihydrate 22.3 mg\nPage 1 of  4   Produced at 12.04.2021 at 08:02:14 AEST\nThis is not an ARTG Certificate document.\nThe onus is on the reader to verify the current accuracy of the information on the document subsequent to the date shown. \nVisit www.tga.gov.au for contact information\nEquivalent: esomeprazole 20 mg\nOther Ingredients (Excipients)', 'Active Ingredients\n \namoxicillin 500 mg S\nOther Ingredients (Excipients)', 'Active Ingredients\nclarithromycin 500 mg\nOther Ingredients (Excipients)']


KeyboardInterrupt: 

In [393]:
out_df

,ARTG ID,Product name,Active ingredients,Sponsor name,Start Date,Dosage Form,Route of Administration,Effective Date,Status
0,98758,AMOXYCILLIN SANDOZ amoxicillin (as trihydrate)...,amoxicillin trihydrate,Sandoz Pty Ltd,3/02/2004,"Tablet, film coated",Oral,9/04/2021,Active
1,98693,AUGMENTIN ES 600/42.9 amoxicillin 600 mg/5mL (...,"amoxicillin trihydrate,potassium clavulanate",Aspen Pharmacare Australia Pty Ltd,2/05/2005,"Suspension, powder for",Oral,25/05/2012,Active
2,97190,"NEXIUM Hp7 Esomeprazole tablet, amoxicillin ca...","amoxicillin,clarithromycin,esomeprazole magnes...",AstraZeneca Pty Ltd,3/12/2003,"Tablet, enteric coated",Oral,18/03/2019,Active
3,93722,MAXAMOX amoxicillin trihydrate 500mg/5mLpowder...,amoxicillin trihydrate,Sandoz Pty Ltd,22/07/2003,"Oral Liquid, powder for",Oral,9/04/2021,Active
4,93721,AMOXYCILLIN SANDOZ amoxicillin trihydrate 125m...,amoxicillin trihydrate,Sandoz Pty Ltd,22/07/2003,"Oral Liquid, powder for",Oral,9/04/2021,Active
5,93720,AMOXYCILLIN SANDOZ amoxicillin trihydrate 250m...,amoxicillin trihydrate,Sandoz Pty Ltd,22/07/2003,"Oral Liquid, powder for",Oral,9/04/2021,Active
6,92765,IBIAMOX amoxicillin 1g (as sodium) powder for ...,amoxicillin,Juno Pharmaceuticals Pty Ltd,21/02/2003,"Injection, powder for",Intravenous,6/07/2020,Active
7,90880,FISAMOX amoxicillin (as sodium) 1000 mg powder...,amoxicillin,Mylan Health Pty Ltd,28/11/2002,"Injection, powder for",Intravenous,10/11/2020,Active
8,82830,CURAM DUO 500/125 amoxicillin 500mg (as trihyd...,"amoxicillin trihydrate,potassium clavulanate",Sandoz Pty Ltd,28/03/2003,"Tablet, film coated",Oral,30/03/2021,Active
9,82829,CURAM DUO FORTE 875/125 amoxicillin 875 mg (as...,"amoxicillin trihydrate,potassium clavulanate",Sandoz Pty Ltd,28/03/2003,"Tablet, film coated",Oral,27/01/2021,Active


In [268]:
## split the resulting active ingredients column into the N columns, one for each ingredient;
## then do the same for the strengths column

out_df_split = pd.concat([out_df["Active ingredients"].str.split(',', expand=True)], axis=1)
out_df_split.columns = [f"Ingredient {i}" for i in range(1, out_df_split.shape[1]+1)]

out_df_split.head()

,Ingredient 1,Ingredient 2,Ingredient 3
0,amoxicillin trihydrate,None,None
1,amoxicillin trihydrate,potassium clavulanate,None
2,amoxicillin,clarithromycin,esomeprazole magnesium trihydrate
3,amoxicillin trihydrate,None,None
4,amoxicillin trihydrate,None,None


In [335]:
test_string_list = ['amoxicillin 80 mg/mL', 'clavulanic acid 11.4 mg/mL']

In [377]:
for i in test_string_list:
    ingredient, strength = extract_strength_from_product_name(i)
    print(ingredient, strength)

amoxicillin  80 mg/mL
clavulanic acid  11.4 mg/mL


In [376]:
x,y = extract_strength_from_product_name('clavulanic acid 11.4 mg/mL')
print(y)

11.4 mg/mL


In [374]:
pattern = '(^\D*)([0-9]*\.?[0-9]+.*[a-zA-Z]?.*[a-zA-Z]+$)'
test_string = 'clavulanic acid 11.4 mg/mL'
test2 = 'amoxicillin 80 mg/mL'

result = re.search(pattern, test2, flags=re.I)
print(result.group(1))

amoxicillin 


In [398]:
test = ['Active Ingredients\nesomeprazole magnesium trihydrate 22.3 mg\nPage 1 of  4   Produced at 12.04.2021 at 08:02:14 AEST\nThis is not an ARTG Certificate document.\nThe onus is on the reader to verify the current accuracy of the information on the document subsequent to the date shown. \nVisit www.tga.gov.au for contact information\nEquivalent: esomeprazole 20 mg\nOther Ingredients (Excipients)', 'Active Ingredients\n \namoxicillin 500 mg S\nOther Ingredients (Excipients)', 'Active Ingredients\nclarithromycin 500 mg\nOther Ingredients (Excipients)']

In [399]:
get_ingredients_and_strengths_from_PDF(test)

TypeError: expected string or bytes-like object

In [409]:
for i in range(len(test)):
               print(test[i].split("\n")[1:-1])

['esomeprazole magnesium trihydrate 22.3 mg', 'Page 1 of  4   Produced at 12.04.2021 at 08:02:14 AEST', 'This is not an ARTG Certificate document.', 'The onus is on the reader to verify the current accuracy of the information on the document subsequent to the date shown. ', 'Visit www.tga.gov.au for contact information', 'Equivalent: esomeprazole 20 mg']
[' ', 'amoxicillin 500 mg S']
['clarithromycin 500 mg']
